# TonlibClient example

In [1]:
import sys
sys.path.insert(0, '/app')

import logging
import requests
import asyncio

from pytonlib import TonlibClient


logging.basicConfig(format='%(asctime)s %(module)-15s %(message)s',
                    level=logging.INFO)

Reading latest config:

In [2]:
ton_config_url = 'https://newton-blockchain.github.io/global.config.json'
ton_config = requests.get(ton_config_url).json()

Creating TonlibClient with single LiteServer. You can select LiteServer by index in config file:

In [3]:
loop = asyncio.get_running_loop()
client = TonlibClient(ls_index=0, # choose LiteServer to connect
                      config=ton_config,
                      keystore='/tmp/ton_keystore',
                      loop=loop)

await client.init()

[ 4][t 1][2022-05-06 12:06:30.259248100][TonlibClient.cpp:1477][!Tonlib][&tonlib_query]	Tonlib got query [id:1] setLogVerbosityLevel {
  new_verbosity_level = 0
}
[ 4][t 1][2022-05-06 12:06:30.259297000][TonlibClient.cpp:1516][!Tonlib][&tonlib_query]	Tonlib got static query setLogVerbosityLevel {
  new_verbosity_level = 0
}
2022-05-06 12:06:30,266 client          TonLib #000 inited successfully


Reading masterchain info:

In [4]:
masterchain_info = await client.get_masterchain_info()
masterchain_info

{'@type': 'blocks.masterchainInfo',
 'last': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 20352855,
  'root_hash': 'HybomVdog/VXwRh/uijnoW4GNyg3uZU/Yf2klSZc3w8=',
  'file_hash': 'e8ODh5TTAk2GbOzGVX5bWYFuw2TOY4g9dnRLp71GIJo='},
 'state_root_hash': '11WdVLe2zjALVR5Vy+zODD+4Z3fUkUMS//+nChBwnTw=',
 'init': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '0',
  'seqno': 0,
  'root_hash': 'F6OpKZKqvqeFp6CQmFomXNMfMj2EnaUSOXN+Mh+wVWk=',
  'file_hash': 'XplPz01CXAps5qeSWUtxcyBfdAo5zVb1N979KLSKD24='},
 '@extra': '1651838801.2616553:0:0.6755289135658142'}

Block header for the last masterchain block:

In [5]:
block_header = await client.get_block_header(**masterchain_info['last'])
block_header

{'@type': 'blocks.header',
 'id': {'@type': 'ton.blockIdExt',
  'workchain': -1,
  'shard': '-9223372036854775808',
  'seqno': 20352855,
  'root_hash': 'HybomVdog/VXwRh/uijnoW4GNyg3uZU/Yf2klSZc3w8=',
  'file_hash': 'e8ODh5TTAk2GbOzGVX5bWYFuw2TOY4g9dnRLp71GIJo='},
 'global_id': -239,
 'version': 0,
 'flags': 1,
 'after_merge': False,
 'after_split': False,
 'before_split': False,
 'want_merge': True,
 'want_split': False,
 'validator_list_hash_short': -567360299,
 'catchain_seqno': 306818,
 'min_ref_mc_seqno': 20352852,
 'is_key_block': False,
 'prev_key_block_seqno': 20351399,
 'start_lt': '27690238000000',
 'end_lt': '27690238000004',
 'gen_utime': 1651838783,
 'vert_seqno': 1,
 'prev_blocks': [{'@type': 'ton.blockIdExt',
   'workchain': -1,
   'shard': '-9223372036854775808',
   'seqno': 20352854,
   'root_hash': 'eYMrWkNTQEZCl1GpzZgNZnu/Pn5H4tDyjuDVELoUhaQ=',
   'file_hash': 'XLaOuSRinsM3nibx7Ohwn0pvNN3Op0bZD0FXC8nUbxA='}],
 '@extra': '1651838801.8503692:0:0.04694758922595088'}

Shards of the masterchain block:

In [6]:
shards = await client.get_shards(master_seqno=masterchain_info['last']['seqno'])
shards

{'@type': 'blocks.shards',
 'shards': [{'@type': 'ton.blockIdExt',
   'workchain': 0,
   'shard': '-9223372036854775808',
   'seqno': 25523717,
   'root_hash': '+vQ9EGj9do3ZaYD45CPx7TNXDEawv6Sn7jm9A/clhUg=',
   'file_hash': 'sejNgFj1dVDgysh+sbTuHaq78hdmJn+MsXlVGIqET6I='}],
 '@extra': '1651838802.874297:0:0.8589753613044604'}

Read transactions from the shard block:

In [7]:
txs = await client.get_block_transactions(**masterchain_info['last'], count=10)

print('Is incomplete:', txs['incomplete'])
print('Num txs:', len(txs['transactions']))

Is incomplete: False
Num txs: 5


Read transaction details:

In [8]:
tx = txs['transactions'][0]
tx

{'@type': 'blocks.shortTxId',
 'mode': 135,
 'account': '-1:3333333333333333333333333333333333333333333333333333333333333333',
 'lt': '27690238000001',
 'hash': 'UOooABBR9GbHy6Xc9XoQrP0O23E9olCvQQfQccAx5oY='}

In [9]:
await client.get_transactions(**tx, limit=1)

[{'@type': 'raw.transaction',
  'address': {'@type': 'accountAddress',
   'account_address': 'Ef8zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzM0vF'},
  'utime': 1651838789,
  'data': 'te6cckECBwEAAYkAA69zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzAAAZLyLrQAJEADAz5R9kyiBp6FDszsyvjxs+S6ZCykpfnwFgXaXSAQAAGS8i60ABYnUPRQABQIAQIDAQGgBACCco4xkVFKupxUGvO41Ydm9foAhJtJVq3G9Zw5u+QyMqwnFqNBt71om/njbC9S5zY+hsJRhXrP+u6e4IRL47UnsOwCDwQJN4DTU5gRBQYAq2n+AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE/zMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzMzTeA01OAAAADJeRdaAAMTqHopAAJ5Cr2wWu+wAAAAAAAAAAGQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFvAAAAAAAAAAAAAAAABLUUtpEnlC4z33SeGHxRhIq/htUa7i3D8ghbwxhQTn44EnHVZsA==',
  'transaction_id': {'@type': 'internal.transactionId',
   'lt': '27690240000002',
   'hash': 'DPXzyS0NpTZjMmcOTpqXsMIvkrqIdoSK/XKP/aZkyUs='},
  'fee': '0',
  'storage_fee': '0',
  'other_fee': '0',
  'in_msg': {'@type': 'raw.message',
   'source': 'Ef8AAAAAAAAAAAAAAAA

In [14]:
headers = []

start_seqno = 20352855
for i in range(1024):
    header_future = client.get_block_header(-1, -9223372036854775808, start_seqno - i)
    headers.append(header_future)
result = await asyncio.gather(*headers, loop=loop)
print(len(result))
result[:2]

/usr/lib/python3.8/asyncio/base_events.py:422: RuntimeWarning: coroutine 'TonlibClient.get_block_header' was never awaited
  return futures.Future(loop=self)


1024


[{'@type': 'blocks.header',
  'id': {'@type': 'ton.blockIdExt',
   'workchain': -1,
   'shard': '-9223372036854775808',
   'seqno': 20352855,
   'root_hash': 'HybomVdog/VXwRh/uijnoW4GNyg3uZU/Yf2klSZc3w8=',
   'file_hash': 'e8ODh5TTAk2GbOzGVX5bWYFuw2TOY4g9dnRLp71GIJo='},
  'global_id': -239,
  'version': 0,
  'flags': 1,
  'after_merge': False,
  'after_split': False,
  'before_split': False,
  'want_merge': True,
  'want_split': False,
  'validator_list_hash_short': -567360299,
  'catchain_seqno': 306818,
  'min_ref_mc_seqno': 20352852,
  'is_key_block': False,
  'prev_key_block_seqno': 20351399,
  'start_lt': '27690238000000',
  'end_lt': '27690238000004',
  'gen_utime': 1651838783,
  'vert_seqno': 1,
  'prev_blocks': [{'@type': 'ton.blockIdExt',
    'workchain': -1,
    'shard': '-9223372036854775808',
    'seqno': 20352854,
    'root_hash': 'eYMrWkNTQEZCl1GpzZgNZnu/Pn5H4tDyjuDVELoUhaQ=',
    'file_hash': 'XLaOuSRinsM3nibx7Ohwn0pvNN3Op0bZD0FXC8nUbxA='}],
  '@extra': '1651839046.64598